# Instructions

Travail individuel à réaliser par chaque étudiant. Chaque fichier devra ensuite être rassemblé par groupe dans le premier dépôt Git de l'année universitaire, dans un nouveau dossier nommé <code>Computer Vision</code>.

Le nom du fichier doit être le prénom de l'étudiant écrit en minuscules. Par exemple, si l'étudiant s'appelle BOB Toto, le fichier doit être nommé toto.ipynb.

# Détails de l'étudiant
### Nom(s)  : RAKOTOHARIMALALA 
### Prénom(s) : Ny Hasina Sedera
### Classe : ESIIA4

# Vision par Ordinateur avec Keras/TensorFlow : Un Notebook Pratique et Conceptuel

Ce notebook a pour objectif de vous guider pas à pas dans la création et l'analyse d'un modèle de réseau de neurones convolutif (CNN) appliqué au jeu de données CIFAR-10. Chaque étape est accompagnée d'explications pratiques ainsi que de questions conceptuelles pour renforcer votre compréhension des enjeux théoriques et pratiques de la vision par ordinateur.

## Étape 1 : Introduction et Configuration de l'Environnement

Dans cette étape, nous allons configurer notre environnement de travail et importer les bibliothèques indispensables pour le deep learning et la manipulation de données. Nous vérifions également la version de TensorFlow pour nous assurer que tout fonctionne correctement.

### Explication Pratique
La bonne configuration de l'environnement est cruciale pour garantir la reproductibilité et la stabilité de vos expériences. En particulier, les versions des bibliothèques peuvent influencer le comportement du modèle et sa performance, d'où l'importance de vérifier et documenter ces versions dès le début.

In [1]:
# Importer les bibliothèques nécessaires
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

print('Version de TensorFlow :', tf.__version__)

ModuleNotFoundError: No module named 'tensorflow'

### Question  1

**Q1 :** Pourquoi est-il essentiel de vérifier la configuration de l'environnement (versions des bibliothèques, dépendances, etc.) avant de développer un modèle de deep learning ?

_Répondez dans une nouvelle cellule Markdown._

Il est essentiel de vérifier la configuration de l'environnement avant de développer un modèle de deep learning pour : 
- **Eviter les incompatibilités** : Des écarts entre les versions des bibliothèques peuvent entraîner des erreurs d’exécution, des comportements inattendus ou une baisse de performance du modèle.
- **Garantir la reproductibilité** : En précisant avec rigueur les versions utilisées, on s’assure d’obtenir des résultats cohérents, quel que soit l’environnement d’exécution.
- **Optimiser l’utilisation du matériel** : Certaines versions de bibliothèques sont spécialement optimisées pour certaines architectures matérielles (CPU, GPU, TPU), améliorant ainsi la rapidité et l’efficacité du traitement.
- **Simplifier la gestion des dépendances** : L’emploi d’environnements virtuels (comme conda ou venv) permet d’isoler les bibliothèques de chaque projet et d’éviter les conflits entre installations.
- **Assurer la stabilité et simplifier la maintenance** : Un environnement bien structuré limite les bugs causés par des mises à jour non maîtrisées et simplifie la gestion du projet sur le long terme.


## Étape 2 : Chargement et Prétraitement des Données

Nous allons charger le jeu de données CIFAR-10, composé de 60 000 images couleur réparties en 10 classes. Dans cette étape, nous normalisons les valeurs des pixels afin qu'elles soient comprises entre 0 et 1, et nous transformons les étiquettes en format one-hot pour faciliter le processus de classification.

### Explication Pratique
La normalisation aide à stabiliser et accélérer l'entraînement du modèle en assurant que les valeurs d'entrée ont une échelle comparable. Le one-hot encoding évite que le modèle interprète les étiquettes comme des valeurs numériques ordonnées, ce qui est essentiel pour les problèmes de classification multi-classes.

In [ ]:
# Charger le jeu de données CIFAR-10
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normaliser les valeurs des pixels (entre 0 et 1)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convertir les vecteurs de classes en matrices binaires (one-hot encoding)
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print("Forme des données d'entrainement :", x_train.shape)
print("Forme des étiquettes d'entraînement :", y_train.shape)

### Question 2

**Q2 :** Expliquez comment la normalisation des pixels et le one-hot encoding des étiquettes contribuent chacun à la stabilité et à l'efficacité de l'entraînement d'un modèle de deep learning.

_Répondez dans une nouvelle cellule Markdown._

### Normalisation des pixels et one-hot encoding des étiquettes  

#### **1. Normalisation des pixels**  
- **Stabilité** : Ramener les valeurs des pixels dans l’intervalle [0,1] réduit les écarts excessifs, évitant ainsi des gradients instables et la saturation des fonctions d’activation. Cela facilite un passage fluide des données dans le réseau, favorisant un apprentissage équilibré. 
- **Efficacité** :  Accélère l’entraînement en optimisant l’ajustement des poids et en stabilisant les algorithmes d’optimisation comme Adam ou SGD. De plus, cette homogénéisation des entrées améliore la généralisation du modèle.
#### **2. One-hot encoding des étiquettes**  
- **Stabilité** :  Convertir les étiquettes en vecteurs binaires empêche le modèle d’inférer une relation d’ordre entre les classes (par exemple, éviter que la classe 2 soit perçue comme plus grande que la classe 1). Cela garantit une meilleure interprétation des catégories.
- **Efficacité** : Améliore la précision de l’apprentissage en rendant possible l’utilisation de fonctions de perte adaptées comme categorical_crossentropy. Cela permet également au réseau de calculer correctement les probabilités pour chaque classe et d’éviter des erreurs de classification dues à un encodage inapproprié des sorties.


## Étape 3 : Exploration et Visualisation des Données

Avant de construire le modèle, il est important d'explorer et de visualiser les données. Nous affichons ainsi un échantillon d'images du jeu de données pour mieux comprendre leur contenu et la distribution des classes.

### Explication Pratique
La visualisation des données permet d'identifier d'éventuelles anomalies, comme des classes sous-représentées ou des images bruitées, et de décider si des techniques d'augmentation de données ou de prétraitement supplémentaires sont nécessaires.

In [ ]:
# Afficher quelques images du jeu de données d'entraînement
noms_classes = ['avion', 'automobile', 'oiseau', 'chat', 'cerf',
               'chien', 'grenouille', 'cheval', 'navire', 'camion']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i])
    plt.xlabel(noms_classes[y_train[i].argmax()])
plt.show()

### Question 3

**Q3 :** D'après la visualisation, discutez de l'impact potentiel d'une distribution inégale des classes ou de la présence d'images de mauvaise qualité sur la performance d'un modèle de classification. Quelles stratégies pourraient être mises en place pour pallier ces problèmes ?

_Répondez dans une nouvelle cellule Markdown._

### Impact d’une distribution inégale des classes et des images de mauvaise qualité sur la performance du modèle

#### **1. Effets d’une distribution inégale des classes**  
  - **Biais du modèle** : Si certaines classes sont sous-représentées, le modèle risque d’être biaisé en faveur des classes majoritaires et d’avoir du mal à bien généraliser sur les classes rares.
  - **Mauvaise généralisation** :  Le modèle pourrait avoir un taux d’erreur plus élevé sur les classes minoritaires, car il n’a pas assez d’exemples pour apprendre leurs caractéristiques distinctives.
  - **Métriques déséquilibrées** : Une forte présence d’une classe dominante peut fausser des indicateurs comme l’accuracy, rendant le modèle apparemment performant alors qu’il ne prédit correctement que les classes les plus fréquentes.

#### **2. Effets des images de mauvaise qualité**  
- **Difficulté d’apprentissage** : Des images floues, bruitées ou mal étiquetées peuvent perturber l’apprentissage du modèle en introduisant des erreurs dans les données.  
- **Mauvaise convergence** :  Si trop d’images sont de mauvaise qualité, le modèle peut apprendre des caractéristiques non pertinentes, ce qui dégrade la performance globale.

### Stratégies pour pallier ces problèmes

 #### **1. Pour équilibrer la distribution des classes**
 - **Rééchantillonnage des données** :
   - Sur-échantillonner les classes sous-représentées (ajout d’exemples par duplication ou synthèse d’images via des techniques comme SMOTE).
   - Sous-échantillonner les classes majoritaires pour réduire l’effet de dominance.

- **Utiliser des métriques adaptées** : Privilégier des métriques comme le F1-score ou l’AUC-ROC au lieu de l’accuracy pour mieux évaluer les performances sur toutes les classes.
- **Pondération des classes** : Attribuer un poids plus important aux classes rares lors de l’entraînement pour compenser leur faible représentation.

   #### **2. Pour améliorer la qualité des images**
   - **Nettoyage des données** : Identifier et supprimer les images mal étiquetées ou trop bruitées.
   - **Techniques de prétraitement** : Appliquer des filtres pour améliorer la netteté et réduire le bruit dans les images.
   - **Augmentation des données** : Utiliser des transformations (rotation, recadrage, ajustement des couleurs) pour enrichir le jeu de données et rendre le modèle plus robuste aux variations visuelles.


## Étape 4 : Construction du Modèle CNN

Nous allons construire un réseau de neurones convolutif (CNN) pour extraire des caractéristiques hiérarchiques des images. Ce modèle se compose de plusieurs blocs de convolution suivis de couches de pooling et se termine par des couches entièrement connectées pour la classification.

### Explication Pratique
Les couches de convolution permettent au modèle de détecter des motifs locaux (comme les contours ou les textures), tandis que les couches de pooling réduisent la dimensionnalité, ce qui diminue la charge computationnelle et aide à rendre le modèle plus robuste aux translations. Le dropout, quant à lui, est une technique de régularisation qui aide à prévenir le surapprentissage en désactivant aléatoirement certains neurones pendant l'entraînement.

In [ ]:
# Construire le modèle CNN
model = models.Sequential()

# Bloc de convolution 1 : 32 filtres, taille 3x3, activation ReLU
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=x_train.shape[1:]))
model.add(layers.MaxPooling2D((2, 2)))

# Bloc de convolution 2 : 64 filtres
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Bloc de convolution 3 : 64 filtres
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Aplatir les sorties et ajouter des couches entièrement connectées
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()

### Question 4

**Q4 :** Décrivez le rôle de chaque composant du CNN (couches de convolution, pooling et dropout) et expliquez comment ils interagissent pour permettre au modèle d'extraire des caractéristiques pertinentes des images.

_Répondez dans une nouvelle cellule Markdown._


#### **Rôle des composants du CNN**  
- **1-Couches de convolution**
    - Détectent les caractéristiques locales (bords, textures, formes).
    - Extraitent des informations de plus en plus complexes à chaque couche.
- **2-Couches de Pooling** :
    - Réduisent la taille des cartes de caractéristiques, limitant la charge computationnelle.
    - Rendent le modèle plus robuste aux variations de position et de taille des objets.
- **2-Couches de Dropout** :
    - Prévenient le surapprentissage en désactivant aléatoirement certains neurones.
    - Améliorent la généralisation du modèle sur de nouvelles images.
  
#### **Interaction entre les composants**  
    - La convolution extrait les caractéristiques, le pooling les simplifie, et le dropout empêche la suradaptation.
    - Ensemble, ces couches permettent au CNN d’analyser efficacement les images et de les classifier avec précision.
    

## Étape 5 : Compilation et Entraînement du Modèle

Nous allons maintenant compiler le modèle en choisissant un optimiseur, une fonction de perte ainsi que des métriques d'évaluation. Ensuite, nous entraînons le modèle sur les données d'entraînement en réservant une partie des données pour la validation.

### Explication Pratique
La compilation configure le processus d'apprentissage, notamment la manière dont les poids seront ajustés via la rétropropagation. Le choix de l'optimiseur (ici, Adam) et la définition des hyperparamètres (comme le taux d'apprentissage et la taille du batch) influencent grandement la vitesse de convergence et la qualité finale du modèle.

In [ ]:
# Compiler le modèle
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(x_train, y_train, epochs=10, batch_size=64,
                    validation_split=0.2)

### Question 5

**Q5 :** Quels sont les effets d'un choix inadapté d'hyperparamètres (comme le taux d'apprentissage ou la taille du batch) sur l'entraînement d'un réseau de neurones ? Expliquez en quoi un optimiseur bien configuré est crucial pour la convergence du modèle.

_Répondez dans une nouvelle cellule Markdown._

#### **Effets d'un choix inadapté d'hyperparamètres**  
- **Taux d'apprentissage** :
    - Un taux trop élevé peut empêcher le modèle de converger correctement en provoquant des fluctuations importantes.
    - Un taux trop faible ralentit l’apprentissage et peut bloquer le modèle dans un minimum sous-optimal. 
- **Taille du batch** :
    - Une taille trop petite rend l'entraînement plus instable en introduisant du bruit.
    - Une taille trop grande peut limiter la capacité du modèle à généraliser et augmenter l'utilisation de mémoire.

#### **Importance d'un optimiseur bien configuré**  
Un optimiseur comme Adam ou SGD, bien paramétré, ajuste efficacement les poids du modèle, favorisant une convergence stable et rapide tout en évitant les erreurs d’optimisation.


## Étape 6 : Évaluation du Modèle

Après l'entraînement, nous évaluons notre modèle sur le jeu de test afin de mesurer sa capacité de généralisation sur des données inédites. Les métriques telles que la perte et la précision nous aident à quantifier la performance globale du modèle.

### Explication Pratique
L'évaluation sur un jeu de test indépendant permet de détecter un éventuel surapprentissage (overfitting). Si le modèle présente une bonne performance sur l'entraînement mais une performance médiocre sur le test, cela indique qu'il n'a pas suffisamment généralisé, ce qui peut nécessiter des ajustements comme plus de régularisation ou des techniques d'augmentation de données.

In [ ]:
# Évaluer le modèle sur le jeu de test
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('Précision sur le jeu de test :', test_acc)

### Question  6

**Q6 :** Que nous indiquent la perte et la précision obtenues lors de l'évaluation sur le jeu de test ? Quels ajustements pourriez-vous envisager si vous observez un écart significatif entre les performances sur l'entraînement et le test ?

_Répondez dans une nouvelle cellule Markdown._

#### **Perte et précision sur le jeu de test**  
- **Perte** : Indique à quel point les prédictions du modèle s’écartent des valeurs réelles. Une perte élevée suggère un mauvais ajustement aux données.
- **Précision** : Mesure la proportion de prédictions correctes. Une précision élevée signifie que le modèle classe bien les images.
#### **Ajustements à envisager en cas d'écart significatif**  
- **Si surapprentissage (overfitting)** → Bonne performance en entraînement mais mauvaise en test : 
  - Ajouter du dropout ou d'autres techniques de régularisation.
  - Utiliser de l’augmentation de données pour améliorer la généralisation.
  - Réduire la complexité du modèle (moins de couches ou de neurones).
- **Si sous-apprentissage (underfitting)** → Mauvaise performance sur les deux ensembles :
  - Entraîner plus longtemps ou ajuster le taux d’apprentissage.
  -Essayer un modèle plus complexe avec plus de couches.
  -Ajouter plus de données d’entraînement si possible.


## Étape 7 : Prédictions et Visualisation des Résultats

Nous allons utiliser le modèle entraîné pour prédire les classes des images du jeu de test. La visualisation des résultats nous permet de comparer les étiquettes prédites aux étiquettes réelles et d'identifier les erreurs potentielles.

### Explication Pratique
La visualisation aide à comprendre qualitativement comment le modèle se comporte face à différentes images. Cela permet d'identifier si certaines classes sont systématiquement mal prédites ou si le modèle confond certaines catégories, ouvrant ainsi la voie à des améliorations ultérieures (par exemple, via l'augmentation de données ou des ajustements de l'architecture).

In [ ]:
# Faire des prédictions sur le jeu de test
predictions = model.predict(x_test)

# Fonction pour afficher l'image avec les étiquettes prédites et réelles
def afficher_image(i, predictions_array, etiquette_vraie, img):
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img, cmap=plt.cm.binary)

    etiquette_predite = np.argmax(predictions_array)
    etiquette_vraie = np.argmax(etiquette_vraie)

    couleur = 'blue' if etiquette_predite == etiquette_vraie else 'red'
    plt.xlabel(f"Prédit : {noms_classes[etiquette_predite]} (Vrai : {noms_classes[etiquette_vraie]})", color=couleur)

# Afficher quelques images de test avec leurs prédictions
nb_lignes = 5
nb_colonnes = 3
nb_images = nb_lignes * nb_colonnes
plt.figure(figsize=(2 * nb_colonnes, 2 * nb_lignes))
for i in range(nb_images):
    plt.subplot(nb_lignes, nb_colonnes, i+1)
    afficher_image(i, predictions[i], y_test[i], x_test[i])
plt.tight_layout()
plt.show()

### Question 7

**Q7 :** Après avoir examiné les prédictions, identifiez et discutez des stratégies conceptuelles (par exemple, l'augmentation de données, le raffinement de l'architecture ou l'ajustement des hyperparamètres) qui pourraient améliorer la robustesse et la précision du modèle.

_Répondez dans une nouvelle cellule Markdown._


#### **Stratégies pour améliorer la robustesse et la précision du modèle**  
- **Augmentation des données** : Générer des variations (rotation, zoom, bruit) pour améliorer la capacité du modèle à généraliser et éviter qu’il se focalise sur des caractéristiques spécifiques. 
- **Optimisation de l'architecture** :Ajouter des couches convolutionnelles, ajuster la taille des filtres ou tester différentes architectures (ResNet, VGG, EfficientNet) pour extraire des caractéristiques plus pertinentes.
- **Ajustement des hyperparamètres** : Optimiser le taux d’apprentissage, la taille du batch ou la fonction d’activation pour améliorer la convergence et la stabilité de l’entraînement.
- **Pondération des classes** : En cas de déséquilibre entre les classes, ajuster leur poids dans la fonction de perte pour éviter que le modèle ne favorise les classes majoritaires et garantir un apprentissage plus équilibré.

Ces stratégies permettent de rendre le modèle plus robuste face aux variations des données et d'améliorer sa précision sur de nouveaux exemples.

## Étape 8 : Conclusion et Travaux Futurs

Dans ce notebook, nous avons :
- Configuré l'environnement et importé les bibliothèques nécessaires
- Chargé et prétraité le jeu de données CIFAR-10
- Exploré et visualisé les données
- Construit, compilé et entraîné un modèle CNN
- Évalué le modèle et visualisé ses prédictions

### Explication Pratique
Ce pipeline offre une approche complète, à la fois pratique et conceptuelle, pour la mise en œuvre d'un modèle de vision par ordinateur. Pour aller plus loin, vous pouvez explorer des architectures plus complexes, appliquer des techniques d'augmentation de données ou encore expérimenter avec différents optimisateurs afin de mieux comprendre l'impact de chacun sur la performance du modèle.